In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import  lightgbm as lgbm

In [3]:
ks=pd.read_csv('G:/MINE/SUB/Thesis/kaggle/6.Feature Engineering/DataSet/ks-projects-201801.csv',parse_dates=['launched','deadline'])

In [10]:
ks.head(7)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,0
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,52375.0,50000.00,1
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,successful,16,US,1205.0,1205.0,1000.00,1


In [5]:
ks=ks.query('state!="live"')

In [7]:
ks=ks.assign(outcome= (ks.state=='successful').astype(int))

In [11]:
ks=ks.assign(day=ks.launched.dt.day,
            month=ks.launched.dt.month,
            year=ks.launched.dt.year,
            hour=ks.launched.dt.hour)

In [14]:
cat_col=['category','currency','country']
label=LabelEncoder()
encode=ks[cat_col].apply(label.fit_transform)
data=ks[['day','month','year','hour','outcome','goal']].join(encode)

In [20]:
def make_split(dataset):
    size=int(len(dataset)*.1)
    train=dataset[:-2*size]
    valid=dataset[-2*size:-size]
    test=dataset[-size:]
    return train,valid,test
def model_evaluation(train,valid,test):
    feature=train.columns.drop('outcome')
    dvalid=lgbm.Dataset(valid[feature],label=valid['outcome'])
    dtrain=lgbm.Dataset(train[feature],label=train['outcome'])
    param={'num_leaves':64,'objective':'binary','metric':'auc'}
    num_round=1000
    model=lgbm.train(param,dtrain,num_round,valid_sets=[dvalid],early_stopping_rounds=10,verbose_eval=False)
    valid_pred=model.predict(valid[feature])
    print('valid_accuracy :',roc_auc_score(valid['outcome'],valid_pred))
    test_pred=model.predict(test[feature])
    print('test_accuracy :',roc_auc_score(test['outcome'],test_pred))
def model_evaluation_two(train,valid,test):
    feature=train.columns.drop('outcome')
    dvalid=lgbm.Dataset(valid[feature],label=valid['outcome'])
    dtrain=lgbm.Dataset(train[feature],label=train['outcome'])
    param={'num_leaves':64,'objective':'binary','metric':'auc','seed':7}
    num_round=1000  #num_boost_round same 
    model=lgbm.train(param,dtrain,num_boost_round=1000,valid_sets=[dvalid],early_stopping_rounds=10,verbose_eval=False)
    valid_pred=model.predict(valid[feature])
    print('valid_accuracy :',roc_auc_score(valid['outcome'],valid_pred))
    test_pred=model.predict(test[feature])
    print('test_accuracy :',roc_auc_score(test['outcome'],test_pred))

In [17]:
#normal label encoding
train,valid,test=make_split(data)
model_evaluation(train,valid,test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013326 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7473111463974601
test_accuracy : 0.7472160532987071


In [19]:
#seed set performance check----result = 'low using seed'
model_evaluation_two(train,valid,test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010929 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7475021559270776
test_accuracy : 0.7468851820348801


In [21]:
#change to me_two train parameter num_round to num_boost_round=1000 and check the outcome 
#compaire the outcome with upper cell
model_evaluation_two(train,valid,test)# result---- num_round work as positional parameter of 'num_boost_round'--result same

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7475021559270776
test_accuracy : 0.7468851820348801


In [29]:
import category_encoders as ce
count_en=ce.CountEncoder()
count_en2=ce.CountEncoder()

In [25]:
count_encode=count_en.fit_transform(ks[cat_col])

In [31]:
#chk=ks[cat_col].apply(count_en2.fit_transform) #encode=ks[cat_col].apply(label.fit_transform)

# not working as we do for label encoding

In [33]:
count_data=data.join(count_encode.add_suffix('_count'))# label encode for ks was reamaining in our data set

In [34]:
count_data.head(7)

,day,month,year,hour,outcome,goal,category,currency,country,category_count,currency_count,country_count
0,11,8,2015,12,0,1000.0,108,5,9,1362,33853,33393
1,2,9,2017,4,0,30000.0,93,13,22,5174,293624,290887
2,12,1,2013,0,0,45000.0,93,13,22,5174,293624,290887
3,17,3,2012,3,0,5000.0,90,13,22,15647,293624,290887
4,4,7,2015,8,0,19500.0,55,13,22,10054,293624,290887
5,26,2,2016,13,1,50000.0,123,13,22,2788,293624,290887
6,1,12,2014,18,1,1000.0,58,13,22,11454,293624,290887


In [41]:
train,valid,test=make_split(count_data)
model_evaluation(train,valid,test) #much great result than all upper cell

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041224 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 722
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7493565559760544
test_accuracy : 0.7499793413369623


In [36]:
d_count_data=count_data.drop(['category','currency','country'],axis=1)
d_count_data.shape

(375862, 9)

In [37]:
train,valid,test=make_split(d_count_data)
model_evaluation(train,valid,test) # after drop label data evaluation on count data the result was good
#than label evaluation but a less from before drop

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024938 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.747939269662227
test_accuracy : 0.7474335773234078


In [ ]:
#target encoding working using target that's why for test or validataion there will be a chance to got 'target leakage'.
#so first learn target encoding from training set then apply the result on validation or test set

In [51]:
#target encoding
#reload train cause we have now drop_train
target_en=ce.TargetEncoder(cols=cat_col) #cols=  if we don't set it all string will be encoded
#but we have label encode data ,then there will be no encode if don't set cols=our_desire_encoded_list
target_en.fit_transform(train[cat_col],train['outcome'])#learning from train ## fit transform also working as fit
train_TE=train.join(target_en.transform(train[cat_col]).add_suffix('_TE'))
valid_TE=valid.join(target_en.transform(valid[cat_col]).add_suffix('_TE'))
test_TE=test.join(target_en.transform(test[cat_col]).add_suffix('_TE'))

In [47]:
train.head()

,day,month,year,hour,outcome,goal,category,currency,country,category_count,currency_count,country_count
0,11,8,2015,12,0,1000.0,108,5,9,1362,33853,33393
1,2,9,2017,4,0,30000.0,93,13,22,5174,293624,290887
2,12,1,2013,0,0,45000.0,93,13,22,5174,293624,290887
3,17,3,2012,3,0,5000.0,90,13,22,15647,293624,290887
4,4,7,2015,8,0,19500.0,55,13,22,10054,293624,290887


In [48]:
train_TE.head()

,day,month,year,hour,outcome,goal,category,currency,country,category_count,currency_count,country_count,category_TE,currency_TE,country_TE
0,11,8,2015,12,0,1000.0,108,5,9,1362,33853,33393,0.360190,0.357122,0.361636
1,2,9,2017,4,0,30000.0,93,13,22,5174,293624,290887,0.384615,0.373392,0.376631
2,12,1,2013,0,0,45000.0,93,13,22,5174,293624,290887,0.384615,0.373392,0.376631
3,17,3,2012,3,0,5000.0,90,13,22,15647,293624,290887,0.412655,0.373392,0.376631
4,4,7,2015,8,0,19500.0,55,13,22,10054,293624,290887,0.302625,0.373392,0.376631


In [52]:
train_TE.head()#check fit_transform result variation

,day,month,year,hour,outcome,goal,category,currency,country,category_count,currency_count,country_count,category_TE,currency_TE,country_TE
0,11,8,2015,12,0,1000.0,108,5,9,1362,33853,33393,0.360190,0.357122,0.361636
1,2,9,2017,4,0,30000.0,93,13,22,5174,293624,290887,0.384615,0.373392,0.376631
2,12,1,2013,0,0,45000.0,93,13,22,5174,293624,290887,0.384615,0.373392,0.376631
3,17,3,2012,3,0,5000.0,90,13,22,15647,293624,290887,0.412655,0.373392,0.376631
4,4,7,2015,8,0,19500.0,55,13,22,10054,293624,290887,0.302625,0.373392,0.376631


In [40]:
help(target_en)

Help on TargetEncoder in module category_encoders.target_encoder object:

class TargetEncoder(sklearn.base.BaseEstimator, category_encoders.utils.TransformerWithTargetMixin)
 |  TargetEncoder(verbose=0, cols=None, drop_invariant=False, return_df=True, handle_missing='value', handle_unknown='value', min_samples_leaf=1, smoothing=1.0)
 |  
 |  Target encoding for categorical features.
 |  
 |  For the case of categorical target: features are replaced with a blend of posterior probability of the target
 |  given particular categorical value and the prior probability of the target over all the training data.
 |  
 |  For the case of continuous target: features are replaced with a blend of the expected value of the target
 |  given particular categorical value and the expected value of the target over all the training data.
 |  
 |  Parameters
 |  ----------
 |  
 |  verbose: int
 |      integer indicating verbosity of the output. 0 for none.
 |  cols: list
 |      a list of columns to enco

In [42]:
train

,day,month,year,hour,outcome,goal,category,currency,country,category_count,currency_count,country_count
0,11,8,2015,12,0,1000.0,108,5,9,1362,33853,33393
1,2,9,2017,4,0,30000.0,93,13,22,5174,293624,290887
2,12,1,2013,0,0,45000.0,93,13,22,5174,293624,290887
3,17,3,2012,3,0,5000.0,90,13,22,15647,293624,290887
4,4,7,2015,8,0,19500.0,55,13,22,10054,293624,290887
...,...,...,...,...,...,...,...,...,...,...,...,...
302891,24,6,2015,9,1,12000.0,113,13,22,22077,293624,290887
302892,23,9,2014,22,0,2000.0,74,13,22,394,293624,290887
302893,3,10,2011,0,0,2500.0,104,13,22,5741,293624,290887
302894,23,7,2015,21,0,2600.0,58,1,3,11454,14830,14624


In [50]:

model_evaluation_two(train_TE,valid_TE,test_TE) 

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 917
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7501163544946668
test_accuracy : 0.7497408662220735


In [53]:
#check for fit_transform
model_evaluation_two(train_TE,valid_TE,test_TE) 

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 917
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7501163544946668
test_accuracy : 0.7497408662220735


In [58]:
#catboost categorical encoding
#------- work as target probability of given value
#--------each row probability calculate from the row before it 
catboost_en=ce.CatBoostEncoder(cols=cat_col)
catboost_en.fit_transform(train[cat_col],train['outcome'])

train_CB=train.join(catboost_en.transform(train[cat_col]).add_suffix('_catBoost'))
valid_CB=valid.join(catboost_en.transform(valid[cat_col]).add_suffix('_catBoost'))
test_CB = test.join(catboost_en.transform(test[cat_col]).add_suffix('_catBoost'))

In [59]:
model_evaluation_two(train_CB,valid_CB,test_CB) 

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015236 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 920
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7489466614316663
test_accuracy : 0.7492964032562113


In [60]:
model_evaluation(train_CB,valid_CB,test_CB) 

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 921
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501
valid_accuracy : 0.7499661822644439
test_accuracy : 0.7493975118775553
